# Exercise

# Importing Libraries

In [629]:
import pandas as pd
from scipy.stats import zscore
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

In [630]:
df=pd.read_csv('D:\\Programming\\Python\\py-master\\ML\\18_PCA\\Exercise\\heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


# Dropping Outliers

In [631]:
df2=df.drop(['ST_Slope','ExerciseAngina','RestingECG','ChestPainType','Sex'], axis='columns')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    df3=pd.DataFrame((np.abs(zscore(df2))>3))
    print(df3[df3['Cholesterol']==True].index.to_numpy())

[ 76 149 616]


In [632]:
a=np.array(df2.columns)
a

array(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'HeartDisease'], dtype=object)

In [633]:
list={}
for i in a:
    list[str(i)]=df3[df3[i]==True].index.to_list()
    list[i].append('/')

In [634]:
list

{'Age': ['/'],
 'RestingBP': [109, 241, 365, 399, 449, 592, 732, 759, '/'],
 'Cholesterol': [76, 149, 616, '/'],
 'FastingBS': ['/'],
 'MaxHR': [390, '/'],
 'Oldpeak': [166, 324, 702, 771, 791, 850, 900, '/'],
 'HeartDisease': ['/']}

In [635]:
final_list=[]
for d in a:
    for k in range(0,len(list[d])):
        if list[d][k]!='/':
            final_list.append(list[d][k])

In [636]:
final_list

[109,
 241,
 365,
 399,
 449,
 592,
 732,
 759,
 76,
 149,
 616,
 390,
 166,
 324,
 702,
 771,
 791,
 850,
 900]

In [637]:
df.drop(final_list,axis='rows', inplace= True)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


# OneHotEncoding Data

In [638]:
df=pd.get_dummies(df,drop_first=True)
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


# Sacling Data

In [639]:
from sklearn.preprocessing import MinMaxScaler
x=df.drop('HeartDisease', axis='columns')
y=df['HeartDisease']
scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x)



# Training Logistic Regression Model

In [640]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(x_scaled,y,test_size=0.1)

In [641]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=900)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8666666666666667

# Finding the best model with most suitable parameters

In [642]:
model_params={
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C':[10,20,30,40,50],
            'kernel':['linear','poly','rbf','sigmoid'],
            'degree':[3,4,5],
            'decision_function_shape':['ovo','ovr']
        }

    },
    'randomforest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[10,20,40,50],
            'criterion':['gini', 'entropy', 'log_loss'],
            }
        },
    'logisticregression':{
        'model':LogisticRegression(max_iter=900),
        'params':{
            'penalty':['l1', 'l2', 'elasticnet', 'none'],
            'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'multi_class':['auto', 'ovr', 'multinomial']
        }
    },
    'gaussianNB':{
        'model':GaussianNB(),
        'params':{}
    },
    'multinomialNB':{
        'model':MultinomialNB(),
        'params':{}
    },
    'decisiontreeclassifier':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini', 'entropy', 'log_loss'],
            'splitter':['best','random']
        }
    }
}

In [643]:
score=[]
for model_name, mp in model_params.items():
    rs=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    rs.fit(X_train,y_train)
    score.append({
        'model':model_name,
        'best_score':rs.best_score_,
        'best_params':rs.best_params_
    })

c:\Python\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
145 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Python\lib\site-packages\sklearn\linear_model\_logistic.py", line 61, in _check_solver
    raise ValueError(
ValueError: Solver newton-cg supports only 'l2' or 'none' penal

In [644]:
df_score=pd.DataFrame(score,columns=['model','best_score','best_params'])

In [645]:
df_score

,model,best_score,best_params
0,svm,0.870232,"{'C': 30, 'decision_function_shape': 'ovo', 'd..."
1,randomforest,0.876413,"{'criterion': 'gini', 'n_estimators': 40}"
2,logisticregression,0.867756,"{'multi_class': 'auto', 'penalty': 'l1', 'solv..."
3,gaussianNB,0.864037,{}
4,multinomialNB,0.849237,{}
5,decisiontreeclassifier,0.800974,"{'criterion': 'log_loss', 'splitter': 'random'}"


Random Forest comes out to be the best model

# Reducing Dimensions of Data

In [646]:
from sklearn.decomposition import PCA
pca=PCA(0.8)
x_pca=pca.fit_transform(x_scaled)
x_pca_final=scaler.fit_transform(x_pca)


In [647]:
pca.explained_variance_ratio_

array([0.30145329, 0.15916859, 0.10317671, 0.09531264, 0.08864394,
       0.06821456])

In [648]:
pca.n_components_

6

In [649]:
X_train, X_test, y_train, y_test=train_test_split(x_pca_final,y,test_size=0.1)

In [650]:
score=[]
for model_name, mp in model_params.items():
    rs=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    rs.fit(X_train,y_train)
    score.append({
        'model':model_name,
        'best_score':rs.best_score_,
        'best_params':rs.best_params_
    })

c:\Python\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
145 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Python\lib\site-packages\sklearn\linear_model\_logistic.py", line 61, in _check_solver
    raise ValueError(
ValueError: Solver newton-cg supports only 'l2' or 'none' penal

In [651]:
df5=pd.DataFrame(score)
df5

,model,best_score,best_params
0,svm,0.861529,"{'C': 50, 'decision_function_shape': 'ovo', 'd..."
1,randomforest,0.856598,"{'criterion': 'log_loss', 'n_estimators': 50}"
2,logisticregression,0.864006,"{'multi_class': 'multinomial', 'penalty': 'l1'..."
3,gaussianNB,0.844207,{}
4,multinomialNB,0.799724,{}
5,decisiontreeclassifier,0.810927,"{'criterion': 'entropy', 'splitter': 'random'}"


Best model comes out to be Logistic Regression